In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

conf = SparkConf() \
    .setAppName('SparkApp') \
    .setMaster('spark://spark:7077') \
    .set("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4,org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.3") \
    .set("spark.sql.shuffle.partitions", "10")
 

sc = SparkContext.getOrCreate(conf=conf)

from pyspark.sql import SQLContext
# Créer un SQLContext pour les opérations SQL
sql_context = SQLContext(sc)

minio_ip_address = "minio"

:: loading settings :: url = jar:file:/opt/conda/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-919d1326-f5a0-4c18-b8ff-1293736bff58;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.3 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.3 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	

In [7]:
%pip install ipyleaflet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 2.2 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from ipyleaflet import Map, basemaps, Marker, Icon, Popup
from ipywidgets import HTML
import time

icon_url = "./avion_icon.png"
icon = Icon(icon_url=icon_url, icon_size=[40, 40])  # Taille ajustable

center = [45.3288, 1.0]

m = Map(center=center, zoom=5)

message2 = HTML()
message2.value = "Hello <b>World</b>"
message2.placeholder = "Some HTML"
message2.description = "Some HTML"

marker = Marker(location=center, icon=icon, draggable=False)
marker.popup = message2

m.add(marker)

display(m)


Map(center=[45.3288, 1.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_o…

KeyboardInterrupt: 

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, window, avg
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType, FloatType

# Kafka configuration
kafka_broker = "kafka1:9092"
kafka_topic = "planes"

# Define schema for the Kafka message
schema = StructType([
    StructField("icao24", StringType(), True),
    StructField("origin_country", StringType(), True),
    StructField("velocity", FloatType(), True),
    StructField("timestamp", StringType(), True),
    StructField("latitude", FloatType(), True),
    StructField("longitude", FloatType(), True)
])

# Read raw data from Kafka
raw_stream = sql_context.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_broker) \
    .option("subscribe", kafka_topic) \
    .option("startingOffsets", "earliest") \
    .load()

df_parsed = raw_stream.select(
    from_json(col("value").cast("string"), schema).alias("data")
).select("data.*")

# # Output rolling average to the console
query = df_parsed.writeStream \
    .outputMode("append") \
    .format("console") \
    .option("truncate", "false") \
    .start()

query.awaitTermination()

25/03/18 10:00:52 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-8501432d-a30a-4c91-9859-5c72757c64d7. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/03/18 10:00:52 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/03/18 10:00:53 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


-------------------------------------------
Batch: 0
-------------------------------------------
+------+--------------+--------+----------+--------+---------+
|icao24|origin_country|velocity|timestamp |latitude|longitude|
+------+--------------+--------+----------+--------+---------+
|408120|VIR6J         |237.16  |1742290470|51.0609 |-11.3686 |
|39de4b|TVF93PY       |236.97  |1742290470|44.7629 |-0.906   |
|88044f|AIQ3135       |81.05   |1742290458|14.0287 |100.6698 |
|39de4c|TVF8223       |222.31  |1742290469|41.2428 |8.7996   |
|7c6b40|JST672        |145.95  |1742290470|-33.7354|150.9534 |
|801640|AIC304        |253.56  |1742290470|27.8209 |78.1996  |
|440da5|EJU91GA       |250.77  |1742290469|36.0498 |23.2263  |
|880454|AIQ606        |221.21  |1742290461|12.2775 |103.4239 |
|80162f|AIC2677       |85.34   |1742290469|28.5108 |77.2511  |
|aa56d8|UAL1348       |247.33  |1742290470|39.4543 |-79.2748 |
|3d10d0|DEEPS         |60.47   |1742290469|50.4913 |10.0944  |
|4b1809|SWR359D      

-------------------------------------------
Batch: 2
-------------------------------------------
+------+--------------+--------+----------+--------+---------+
|icao24|origin_country|velocity|timestamp |latitude|longitude|
+------+--------------+--------+----------+--------+---------+
|440da6|JAR37         |142.1   |1742292060|43.5249 |7.4157   |
|4b1806|SWR531C       |192.71  |1742292060|45.6436 |6.5511   |
|4b1809|SWR359D       |168.99  |1742292060|48.2359 |8.6008   |
|4b1800|SWR339H       |192.88  |1742292060|48.0445 |7.2595   |
|4952cd|TAP552N       |212.03  |1742292060|46.9534 |7.6021   |
|a1fb3f|N227CL        |1.8     |1742291754|47.6694 |9.5128   |
|39de44|TVF8965       |227.08  |1742292060|40.261  |9.1655   |
|3c664f|DLH6LL        |235.94  |1742292060|49.4129 |9.7021   |
|3d23a3|DELTB         |27.96   |1742292016|49.9369 |9.0469   |
|4b187c|SWR15X        |0.0     |1742291795|47.4607 |8.5554   |
|44b2cd|OOLVM         |44.72   |1742292060|50.9803 |5.5392   |
|3c5eed|EWG73P       

-------------------------------------------
Batch: 4
-------------------------------------------
+------+--------------+--------+----------+--------+---------+
|icao24|origin_country|velocity|timestamp |latitude|longitude|
+------+--------------+--------+----------+--------+---------+
|440da6|JAR37         |142.1   |1742292060|43.5243 |7.4151   |
|4b1806|SWR531C       |192.71  |1742292061|45.6422 |6.5525   |
|4b1809|SWR359D       |168.39  |1742292061|48.2342 |8.6003   |
|4b1800|SWR339H       |192.4   |1742292061|48.0432 |7.2602   |
|4952cd|TAP552N       |212.03  |1742292061|46.9545 |7.6041   |
|39de61|22ST          |34.21   |1742291911|46.4011 |5.7836   |
|440dc2|OEFYI         |89.0    |1742292061|48.8987 |9.3604   |
|44028c|EJU71AL       |171.84  |1742292061|45.5771 |5.4073   |
|3c6663|DLH25Y        |234.45  |1742292061|48.5164 |8.0638   |
|3c6675|DLH4HC        |220.4   |1742292061|49.6501 |9.4613   |
|4b187c|SWR15X        |0.0     |1742291795|47.4607 |8.5554   |
|44b2cd|OOLVM        

-------------------------------------------
Batch: 6
-------------------------------------------
+------+--------------+--------+----------+--------+---------+
|icao24|origin_country|velocity|timestamp |latitude|longitude|
+------+--------------+--------+----------+--------+---------+
|4b1818|SWR5HZ        |12.35   |1742292080|47.4518 |8.5562   |
|39de4c|TVF8223       |228.02  |1742292080|44.2494 |7.0677   |
|440da6|JAR37         |138.7   |1742292080|43.5044 |7.3951   |
|4b1800|SWR339H       |191.73  |1742292080|48.013  |7.2779   |
|4952cd|TAP552N       |210.58  |1742292080|46.977  |7.6456   |
|440dc2|OEFYI         |89.51   |1742292080|48.8987 |9.3368   |
|4b187c|SWR15X        |0.0     |1742291795|47.4607 |8.5554   |
|3c5ef2|EWG4NA        |229.71  |1742292080|49.0005 |9.1344   |
|46b824|AEE5DL        |232.72  |1742292080|50.7047 |9.2653   |
|3ffba7|DMMZH         |39.57   |1742292080|49.471  |8.3167   |
|3d124d|DEFEJ         |51.24   |1742292080|48.0639 |9.3508   |
|4aca8c|NSZ2988      

-------------------------------------------
Batch: 8
-------------------------------------------
+------+--------------+--------+----------+--------+---------+
|icao24|origin_country|velocity|timestamp |latitude|longitude|
+------+--------------+--------+----------+--------+---------+
|440da6|JAR37         |137.68  |1742292089|43.4956 |7.3862   |
|4b1806|SWR531C       |194.98  |1742292089|45.6029 |6.5938   |
|4b1800|SWR339H       |191.92  |1742292089|47.9982 |7.2866   |
|39de44|TVF8965       |226.83  |1742292089|40.3186 |9.1467   |
|3d23a3|DELTB         |27.96   |1742292016|49.9369 |9.0469   |
|3c6663|DLH25Y        |236.57  |1742292089|48.4608 |8.0345   |
|3c6675|DLH4HC        |223.85  |1742292089|49.6251 |9.5383   |
|4b187c|SWR15X        |0.0     |1742291795|47.4607 |8.5554   |
|4b187a|SWR4L         |69.9    |1742292089|47.5058 |8.5038   |
|3c65c5|DLH5H         |93.93   |1742292089|49.9816 |8.2657   |
|ae0133|DUKE61        |158.87  |1742292089|50.4655 |7.7157   |
|3c65d8|DLH61H       

-------------------------------------------
Batch: 10
-------------------------------------------
+------+--------------+--------+----------+--------+---------+
|icao24|origin_country|velocity|timestamp |latitude|longitude|
+------+--------------+--------+----------+--------+---------+
|39de4a|TVF8393       |229.62  |1742292091|40.249  |6.4124   |
|39de4c|TVF8223       |228.81  |1742292091|44.2707 |7.0569   |
|440da6|JAR37         |137.37  |1742292091|43.4937 |7.3843   |
|4b1806|SWR531C       |194.98  |1742292091|45.5998 |6.5971   |
|4b1809|SWR359D       |158.9   |1742292091|48.1904 |8.5872   |
|4952cd|TAP552N       |209.14  |1742292091|46.9899 |7.6695   |
|39de61|22ST          |34.21   |1742291911|46.4011 |5.7836   |
|440dc2|OEFYI         |89.51   |1742292091|48.8987 |9.3237   |
|39de44|TVF8965       |226.83  |1742292091|40.3225 |9.1454   |
|44028c|EJU71AL       |164.62  |1742292090|45.5773 |5.4688   |
|44b2cd|OOLVM         |46.03   |1742292091|50.9855 |5.5217   |
|3c65c5|DLH5H       

-------------------------------------------
Batch: 12
-------------------------------------------
+------+--------------+--------+----------+--------+---------+
|icao24|origin_country|velocity|timestamp |latitude|longitude|
+------+--------------+--------+----------+--------+---------+
|4b1818|SWR5HZ        |13.38   |1742292100|47.4498 |8.558    |
|39de4c|TVF8223       |228.81  |1742292101|44.29   |7.0471   |
|440da6|JAR37         |135.52  |1742292101|43.4837 |7.3743   |
|39de61|22ST          |34.21   |1742291911|46.4011 |5.7836   |
|440dc2|OEFYI         |90.03   |1742292101|48.8987 |9.3113   |
|3c664f|DLH6LL        |234.09  |1742292101|49.3605 |9.8069   |
|4b187c|SWR15X        |0.0     |1742291795|47.4607 |8.5554   |
|44b2cd|OOLVM         |46.71   |1742292101|50.9872 |5.5157   |
|3c65c5|DLH5H         |92.79   |1742292101|49.985  |8.28     |
|3c65cc|DLH6UW        |12.35   |1742292100|51.293  |6.7803   |
|3c65cb|DLH2RW        |227.76  |1742292101|50.2422 |7.1666   |
|3c5ef2|EWG4NA      

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/conda/lib/python3.12/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.12/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.12/socket.py", line 720, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [5]:
query.stop()